In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

In [2]:
def square(x):
  return tf.multiply(x, x)

In [13]:
#diff for squre
grad = tfe.gradients_function(square)

In [12]:
print(square(3.))  # => 9.0
print(grad(3.))    # => [6.0]

tf.Tensor(9.0, shape=(), dtype=float32)
[<tf.Tensor: id=68, shape=(), dtype=float32, numpy=6.0>]


In [5]:
# square의 2차 미분:
gradgrad = tfe.gradients_function(lambda x: grad(x)[0])
gradgrad(3.)  # => [2.0]

[<tf.Tensor: id=21, shape=(), dtype=float32, numpy=2.0>]

In [6]:
# 3차 미분은 None:
gradgradgrad = tfe.gradients_function(lambda x: gradgrad(x)[0])
gradgradgrad(3.)  # => [None]

[None]

In [7]:
# 흐름 제어를 곁들여보면:
def abs(x):
  return x if x > 0. else -x

grad = tfe.gradients_function(abs)

In [8]:
grad(3.)

[<tf.Tensor: id=6, shape=(), dtype=float32, numpy=1.0>]

In [9]:
grad(-3.)

[<tf.Tensor: id=48, shape=(), dtype=float32, numpy=-1.0>]

#### Custom gradients

맞춤형 경사는 eager mode와 graph mode에서 경사를 오버라이드 하는 가장 쉬운 방법 <br>
전방 함수에서는 입력, 출력, 중간 결과에 대한 경사를 정의.

In [14]:
@tf.custom_gradient
def clip_gradient_by_norm(x, norm):
  y = tf.identity(x)
  def grad_fn(dresult):
    return [tf.clip_by_norm(dresult, norm), None]
  return y, grad_fn

In [15]:
def log1pexp(x):
  return tf.log(1 + tf.exp(x))

In [16]:
grad_log1pexp = tfe.gradients_function(log1pexp)

In [18]:
# x = 0 일때 경사 계산은 잘 수행된다.
grad_log1pexp(0.)  # => [0.5]

[<tf.Tensor: id=77, shape=(), dtype=float32, numpy=0.5>]

In [19]:
# 하지만, x = 100일때 수치적으로 불안정해지므로, 계산이 수행되지 않는다.
grad_log1pexp(100.)  # => [nan]

[<tf.Tensor: id=86, shape=(), dtype=float32, numpy=nan>]

여기서 log1pexp함수는 분석적으로 맞춤형 경사로 단순화 시킬 수 있습니다.<br>
아래 구현은 전방향으로 계산될 때 불필요한 계산을 제거함으로서 더 효율적으로 변경된 tf.exp(x)의 값을 잘 사용됩니다.

In [22]:
@tf.custom_gradient
def log1pexp(x):
  e = tf.exp(x)
  def grad(dy):
    return dy * (1 - 1 / (1 + e))
  return tf.log(1 + e), grad
grad_log1pexp = tfe.gradients_function(log1pexp)

In [23]:
# 이전 예제와 마찬가지로, x = 0 일때 계산이 잘 수행됩니다.
grad_log1pexp(0.)  # => [0.5]

[<tf.Tensor: id=96, shape=(), dtype=float32, numpy=0.5>]

In [24]:
# 이제 x = 100 일때도 계산이 잘 수행됩니다.
grad_log1pexp(100.)  # => [1.0]

[<tf.Tensor: id=106, shape=(), dtype=float32, numpy=1.0>]